# Juice Problem ILP

### Author: [Ben Rosenberg](https://benrosenberg.info)

### Imports

We begin by importing some relevant libraries. We import `ortools` to formulate and solve the ILP, and we import `time` to time the entire process of supplying the constraints solving the ILP. Lastly, we'll use `pandas` to work with the input data to the ILP.

In [1]:
from ortools.linear_solver import pywraplp as OR
import time
import pandas as pd

## Problem definition

The "Juice Problem" is pretty vague. This is basically just a simple example of an ILP with constraints based on nutritional values of various ingredients. We'll define an input here:

Ingredient | Calories | Fat | Cholesterol | Sodium | Sugar
-|-|-|-|-|-
A|110| 4|30|340|15
B|150| 8|40|120|35
C|250| 2|60|450|40
D|270|10|20|240|60
E|350|15|70|760|45

And let's make some constraints. Our juice can have:

 - No fewer than 2000 units of calories
 - No more than 70 units of fat
 - No more than 750 units of cholesterol
 - No more than 3000 units of sodium
 
Furthermore, let's say that the ratio of cholesterol to sodium must be at most 1:10.
 
Lastly, let's make the objective to minimize the amount of sugar.

### Input entry

We'll use `pandas` to store our input in a more manageable form (a DataFrame). This allows this notebook to also serve as a minimal example of how one might use the `pandas` library.

In [2]:
# define ingredients using a dictionary
data = {
    'Ingredients' : ['A', 'B', 'C', 'D', 'E'],
    'Calories' : [110, 150, 250, 270, 350],
    'Fat' : [4, 8, 2, 10, 15],
    'Cholesterol' : [30, 40, 60, 20, 70],
    'Sodium' : [340, 120, 450, 240, 760],
    'Sugar' : [15, 35, 40, 60, 45]
}

df = pd.DataFrame.from_dict(data).set_index('Ingredients')

In [3]:
df.head()

,Calories,Fat,Cholesterol,Sodium,Sugar
Ingredients,,,,,
A,110,4,30,340,15
B,150,8,40,120,35
C,250,2,60,450,40
D,270,10,20,240,60
E,350,15,70,760,45


In [4]:
print(df['Calories']['A'])

110


In [14]:
start_time = time.time()

m = OR.Solver('Juice Problem', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

# add variables for each ingredient
I = {}
for ingredient in df.index:
    I[ingredient] = m.NumVar(0, m.infinity(), ingredient)

# add constraint on calories
m.Add(sum(I[ingredient] * df['Calories'][ingredient] for ingredient in df.index) >= 2000)

# add constraint on fat
m.Add(sum(I[ingredient] * df['Fat'][ingredient] for ingredient in df.index) <= 70)

# add constraint on cholesterol
m.Add(sum(I[ingredient] * df['Cholesterol'][ingredient] for ingredient in df.index) <= 750)

# add constraint on sodium
m.Add(sum(I[ingredient] * df['Sodium'][ingredient] for ingredient in df.index) <= 3000)

# add constraint on cholesterol:sodium ratio
m.Add(sum(I[ingredient] * df['Cholesterol'][ingredient] for ingredient in df.index) * 10 
   <= sum(I[ingredient] * df['Sodium'][ingredient] for ingredient in df.index))
    
# set objective
m.Minimize(sum(I[ingredient] * df['Sugar'][ingredient] for ingredient in df.index))

m.Solve()

end_time = time.time()

diff = time.gmtime(end_time - start_time)
print('\n[Total time used: {} minutes, {} seconds]'.format(diff.tm_min, diff.tm_sec))

print('Objective:', m.Objective().Value())


[Total time used: 0 minutes, 0 seconds]
Objective: 356.44711968958313


Now that we have our objective, let's print out our solution values:

In [15]:
for ingredient in df.index:
    print('{} units of {}'.format(
        I[ingredient].solution_value(), ingredient
    ))

0.0 units of A
0.0 units of B
1.1600835737737498 units of C
3.5668092727091842 units of D
2.1341160083573794 units of E


### Integer Program

Say that we have the additional constraint that we can only use integer amounts of each ingredient. This means we need to use integer variables (`m.IntVar(...)`) rather than continuous ones (`m.NumVar(...)`):

In [12]:
start_time = time.time()

m = OR.Solver('Juice Problem', OR.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

# add variables for each ingredient
I = {}
for ingredient in df.index:
    I[ingredient] = m.IntVar(0, m.infinity(), ingredient) # <-- change

# add constraint on calories
m.Add(sum(I[ingredient] * df['Calories'][ingredient] for ingredient in df.index) >= 2000)

# add constraint on fat
m.Add(sum(I[ingredient] * df['Fat'][ingredient] for ingredient in df.index) <= 70)

# add constraint on cholesterol
m.Add(sum(I[ingredient] * df['Cholesterol'][ingredient] for ingredient in df.index) <= 750)

# add constraint on sodium
m.Add(sum(I[ingredient] * df['Sodium'][ingredient] for ingredient in df.index) <= 3000)

# add constraint on cholesterol:sodium ratio
m.Add(sum(I[ingredient] * df['Cholesterol'][ingredient] for ingredient in df.index) * 10 
   <= sum(I[ingredient] * df['Sodium'][ingredient] for ingredient in df.index))
    
# set objective
m.Minimize(sum(I[ingredient] * df['Sugar'][ingredient] for ingredient in df.index))

m.Solve()

end_time = time.time()

diff = time.gmtime(end_time - start_time)
print('\n[Total time used: {} minutes, {} seconds]'.format(diff.tm_min, diff.tm_sec))

print('Objective:', m.Objective().Value())


[Total time used: 0 minutes, 0 seconds]
Objective: 430.0


And here are the solution values:

In [16]:
for ingredient in df.index:
    print('{} units of {}'.format(
        I[ingredient].solution_value(), ingredient
    ))

0.0 units of A
0.0 units of B
1.1600835737737498 units of C
3.5668092727091842 units of D
2.1341160083573794 units of E


Note how adding the constraint that the amounts of each ingredient must be integer results in a higher (worse) objective value. In general, adding constraints to a problem will result in an equal or worse result than the previously obtained one.